In [1]:
import pandas as pd
import logging
import sys
from functions import *
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_distances
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
import os
from langchain.llms import HuggingFacePipeline
from dotenv import load_dotenv
from functions import *
import warnings
warnings.filterwarnings("ignore")

c:\Users\Ignacia\Desktop\Ignacia\WORKBENCH\Chatbot-JS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv() 
api_key = os.getenv('OPENAI_API_KEY')
chat = OpenAI(temperature=0.4, model_name='gpt-3.5-turbo-16k', api_key=api_key)

In [3]:
document = PyPDFLoader("../documentation/documents/JavaScript.pdf")
documents = function_text_splitter(document, 500, 250)

In [4]:
data = pd.read_pickle("../documentation/documentation_JS.pickle")

In [5]:
data.head(2)

,page_content,metadata,embeddings_content
0,EloquentJavaScript\n3raedición\nMarijn Haverbeke,"{'source': './documents/JavaScript.pdf', 'page...","[[-0.49793908, -0.3782933, 0.8006307, -0.02384..."
1,Copyright © 2018 por Marijn Haverbeke\nEste li...,"{'source': './documents/JavaScript.pdf', 'page...","[[-0.15308928, -0.4013646, 0.35679892, 0.16363..."


In [22]:
p = "Para que me sirve aprender javascript?"

In [23]:
docs_ = function_main_documents(p, data, documents)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /bert-base-multilingual-cas

In [24]:

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Definición del prompt
template = """
    Eres Ali, un asistente virtual de personalidad extremadamente amable, tu porposito es ayudar con preguntas de JavaScript
    y manejas la siguiente información: {context} para contestar la
    siguiente pregunta:{human_input} y tienes el siguiente contexto de la conversación {chat_history}.
    Responde solo en español destacando tu personalidad y utilizando solo la información que recibes.
    Contesta de forma detallada.
    Si no sabes la respuesta, sólo dilo."""
prompt = PromptTemplate(
        input_variables=["chat_history", "human_input", "context"], template=template
        )
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
                    # Se crea la cadena
chain = load_qa_chain(
chat, chain_type="stuff", memory=memory, prompt=prompt)
                    
respuesta = chain({"input_documents": docs_,
                    "human_input": p}, return_only_outputs=True)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "\\n    Eres Ali, un asistente virtual de personalidad extremadamente amable, tu porposito es ayudar con preguntas de JavaScript\\n    y manejas la siguiente informaci\\u00f3n: \\u201cNosotros creemos que estamos creando el sistema para nuestros\\npropios prop\\u00f3sitos. Creemos que lo estamos haciendo a nuestra\\npropia imagen... Pero la computadora no

In [25]:
respuesta["output_text"]

'Aprender JavaScript es muy útil porque te permite crear aplicaciones web interactivas y dinámicas. Con JavaScript, puedes agregar funcionalidades a tu sitio web, como formularios interactivos, animaciones, validaciones de datos y mucho más. También puedes crear juegos y aplicaciones móviles utilizando frameworks como React Native.\n\nAdemás, JavaScript es uno de los lenguajes de programación más populares y demandados en la industria. Aprender JavaScript te brinda la oportunidad de conseguir empleo como desarrollador web o incluso trabajar como freelancer. Es un lenguaje versátil y utilizado en muchos proyectos, por lo que tener conocimientos en JavaScript te abrirá muchas puertas en el mundo laboral.\n\nAprender JavaScript también te ayuda a desarrollar habilidades de pensamiento lógico y resolución de problemas. La programación requiere de un enfoque analítico y creativo para encontrar soluciones eficientes a los problemas. A medida que te familiarices con JavaScript, desarrollarás 

In [ ]:
def function_main_app(p:str, data:pd.DataFrame, documents:list):
    docs_ = function_main_documents(p, data, documents)
    # Definición del prompt
    template = """
        Eres Ali, un asistente virtual de personalidad extremadamente amable, tu porposito es ayudar con preguntas de JavaScript
        y manejas la siguiente información: {context} para contestar la
        siguiente pregunta:{human_input} y tienes el siguiente contexto de la conversación {chat_history}.
        Responde solo en español destacando tu personalidad y utilizando solo la información que recibes.
        Contesta de forma detallada.
        Si no sabes la respuesta, sólo dilo."""
    prompt = PromptTemplate(
            input_variables=["chat_history", "human_input", "context"], template=template
            )
    memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
                        # Se crea la cadena
    chain = load_qa_chain(
    chat, chain_type="stuff", memory=memory, prompt=prompt)
                        
    response = chain({"input_documents": docs_,
                        "human_input": p}, return_only_outputs=True)
    return response["output_text"]